In [7]:
import pandas as pd

In [8]:
mon_lst = []
for y in range(1990, 2024):
    for m in ['01-31', '02-29' if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0) else '02-28', '03-31', '04-30', '05-31', '06-30', '07-31', '08-31', '09-30', '10-31', '11-30', '12-31']:
        mon_lst.append(f'{y}-{m}')
mon_df = pd.DataFrame(mon_lst[:-7], columns=['交易月份'])
mon_df['月份'] = mon_df['交易月份'].apply(lambda x: x[:7])
stk_df = pd.read_csv('../../data/csmar/公司文件.csv')
stk_df['on'] = 0
mon_df['on'] = 0
header = pd.merge(stk_df, mon_df, on=['on'], how='left')[['证券代码', '交易月份', '月份']]

In [9]:
df1 = pd.read_csv('../../data/csmar/周_周个股回报率文件.csv', usecols=['证券代码', '周开盘日期', '不考虑现金红利再投资的周个股回报率'])
df1 = df1.sort_values(by=['证券代码', '周开盘日期'])
df1['月份'] = df1['周开盘日期'].apply(lambda x: x[:7])
df1['平均周个股回报率'] = df1[['证券代码', '不考虑现金红利再投资的周个股回报率']].groupby('证券代码').rolling(3*52).mean().reset_index()['不考虑现金红利再投资的周个股回报率']
df1['周个股回报率残差'] = df1['不考虑现金红利再投资的周个股回报率'] - df1['平均周个股回报率']
df1['idiovol'] = df1[['证券代码', '周个股回报率残差']].groupby('证券代码').rolling(3*52).std().reset_index()['周个股回报率残差']
df2 = df1[['证券代码', '月份', 'idiovol']]
df3 = df2.drop_duplicates(subset=['证券代码', '月份'], keep='last')
df4 = pd.merge(header, df3, on=['证券代码', '月份'], how='left')
df5 = df4[['证券代码', '交易月份', 'idiovol']]
df5.to_csv('../../output/csmar/月_35.csv', index=False, encoding='utf-8-sig')
df5

,证券代码,交易月份,idiovol
0,1,1990-01-31,NaN
1,1,1990-02-28,NaN
2,1,1990-03-31,NaN
3,1,1990-04-30,NaN
4,1,1990-05-31,NaN
...,...,...,...
2197475,900957,2023-01-31,0.053724
2197476,900957,2023-02-28,0.053311
2197477,900957,2023-03-31,0.052967
2197478,900957,2023-04-30,0.052359
